In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


batch_size = 32

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_train)

trainset, validationset = random_split(dataset, [40000, 10000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=8,pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [14]:
def compute_accuracy(net, dataloader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    
    for data in dataloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def numel(m: torch.nn.Module, only_trainable: bool = False):
    """
    returns the total number of parameters used by `m` (only counting
    shared parameters once); if `only_trainable` is True, then only
    includes parameters with `requires_grad = True`
    """
    parameters = m.parameters()
    if only_trainable:
        parameters = list(p for p in parameters if p.requires_grad)
    unique = dict((p.data_ptr(), p) for p in parameters).values()
    return sum(p.numel() for p in unique)


class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*16*16, 1200)
        self.bn9 = nn.BatchNorm1d(1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.bn10 = nn.BatchNorm1d(1200)
        self.fc3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = (F.relu(self.bn1(self.conv1(x))))
        x = (F.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.conv2(x))
        
        torch.nn.Dropout(p=0.2, inplace=False)
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
       
        x = F.relu(self.bn9(self.fc1(x)))
        x = F.relu(self.bn10(self.fc2(x)))
        torch.nn.Dropout(p=0.42, inplace=False)
        x =self.fc3(x)
        return x
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 76, 3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(76)
        self.conv2 = nn.Conv2d(76, 76, 3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(76)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(76, 126, 3,stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(126)
        self.conv4 = nn.Conv2d(126, 126, 3,stride=1,padding=1)
        self.bn4 = nn.BatchNorm2d(126)
        self.conv5 = nn.Conv2d(126, 148, 3,stride=1,padding=1)
        self.bn5 = nn.BatchNorm2d(148)
        self.conv6 = nn.Conv2d(148, 148, 3,stride=1,padding=1)
        self.bn6 = nn.BatchNorm2d(148)
        self.conv7 = nn.Conv2d(148, 148, 3,stride=1,padding=1)
        self.bn7 = nn.BatchNorm2d(148)
        self.conv8 = nn.Conv2d(148, 148, 3,stride=1,padding=1)
        self.bn8 = nn.BatchNorm2d(148)
        self.fc1 = nn.Linear(148 *4*4, 1200)
        self.bn9 = nn.BatchNorm1d(1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.bn10 = nn.BatchNorm1d(1200)
        self.fc3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = (F.relu(self.bn1(self.conv1(x))))
        x = (F.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.conv2(x))
        torch.nn.Dropout(p=0.2, inplace=False)
        
        
        x = (F.relu(self.bn3(self.conv3(x))))
        x = (F.relu(self.bn4(self.conv4(x))))
        x = self.pool(self.conv4(x))
        torch.nn.Dropout(p=0.3, inplace=False)
        
        
        x = (F.relu(self.bn5(self.conv5(x))))
        x = (F.relu(self.bn6(self.conv6(x))))
        x = (F.relu(self.bn7(self.conv7(x))))
        x = (F.relu(self.bn8(self.conv8(x))))
        x = self.pool(self.conv8(x))
        torch.nn.Dropout(p=0.37, inplace=False)
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.bn9(self.fc1(x)))
        x = F.relu(self.bn10(self.fc2(x)))
        torch.nn.Dropout(p=0.42, inplace=False)
        x =self.fc3(x)
        return x

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 32
        self.conv = conv3x3(3, 32)
        self.bn = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 128, layers[0])
        self.layer2 = self.make_layer(block, 256*2, layers[1], 2)
        self.layer3 = self.make_layer(block, 256*2, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(256*2, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
    
class LinearBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearBlock, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)
    
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        x = self.bn(x)
        
        return x

class LinearNet(nn.Module):
    def __init__(self, input_dim ,output_dim, layers, num_classes=10):
        super(LinearNet, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = layers
        self.feed = self.make_layers()

    def make_layers(self):
        network_layers = []
        network_layers.append(
            LinearBlock(self.input_dim, self.layers[0])
        )
        
        for i in range(len(self.layers) - 1):
            network_layers.append(
                LinearBlock(self.layers[i], self.layers[i+1])
            )
            
        network_layers.append(
            nn.Linear(self.layers[-1], self.output_dim)
        )
        

        return nn.Sequential(*network_layers)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.feed(x)
        return x    
    

In [42]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

#teacher = ResNet(ResidualBlock, [4, 6, 8]).to(device)

epochs_done = 0
teacher = Net().to(device)
teacher.load_state_dict(torch.load("./Teacher260"))
student = LinearNet(3*32*32, 10, [2048,2048] ).to(device)
#student = LinearNet(3*32*32, 10, [512,512,512,512] ).to(device)
#student.load_state_dict(torch.load("./Shallow2150"))

criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters())#, #momentum=0.9)
epochs_done=0
print(numel(student)/10**6)

train_accuracy = []
validation_accuracy = []

validation_loss = []
train_loss = []
print(student)

10.518538
LinearNet(
  (feed): Sequential(
    (0): LinearBlock(
      (fc): Linear(in_features=3072, out_features=2048, bias=True)
      (bn): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinearBlock(
      (fc): Linear(in_features=2048, out_features=2048, bias=True)
      (bn): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Linear(in_features=2048, out_features=10, bias=True)
  )
)


In [ ]:
#Training Loop
for epoch in range(epochs_done, epochs_done + 150):  # loop over the dataset multiple times
    epochs_done += 1 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, _ = data
        
        inputs = inputs.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = student(inputs)
        with torch.no_grad():
            teacher_outputs = teacher(inputs)
            
        loss = criterion(outputs, teacher_outputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if epoch % 10 == 0 and epoch!=0 :
        torch.save(student.state_dict(), f"./distillation1_{epoch}")
        
        
    if epoch % 5 == 0:
        with torch.no_grad(): 
            v_loss = 0.0
            for j, data in enumerate(validationloader, 0):
                inputs, _ = data

                inputs = inputs.to(device)
                teacher_outputs = teacher(inputs)
                
                outputs = student(inputs)
                v_loss += criterion(outputs, teacher_outputs).item()
                
            validation_loss.append(v_loss/j)
            
            v_accuracy = compute_accuracy(student, validationloader)
            validation_accuracy.append(v_accuracy)
            
            train_loss.append(running_loss/i)
            t_accuracy = compute_accuracy(student, trainloader)
            train_accuracy.append(t_accuracy)
    
    print(f"Epoch: {epoch}, training loss: {running_loss/i}, validation loss: {v_loss/j}, validation accuracy: {v_accuracy}%")
                
print('Finished Training')

Epoch: 0, training loss: 396.967466746835, validation loss: 323.28457147647174, validation accuracy: 15.27%
Epoch: 1, training loss: 332.08962469047503, validation loss: 323.28457147647174, validation accuracy: 15.27%
Epoch: 2, training loss: 313.1781912101947, validation loss: 323.28457147647174, validation accuracy: 15.27%
Epoch: 3, training loss: 302.7493167751975, validation loss: 323.28457147647174, validation accuracy: 15.27%
Epoch: 4, training loss: 300.7344202021774, validation loss: 323.28457147647174, validation accuracy: 15.27%
Epoch: 5, training loss: 294.73693547122855, validation loss: 291.6745228400597, validation accuracy: 16.86%
Epoch: 6, training loss: 290.5726287206523, validation loss: 291.6745228400597, validation accuracy: 16.86%
Epoch: 7, training loss: 284.4148837136115, validation loss: 291.6745228400597, validation accuracy: 16.86%
Epoch: 8, training loss: 280.1985004263176, validation loss: 291.6745228400597, validation accuracy: 16.86%
Epoch: 9, training los

In [41]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = student(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 24 %


In [40]:
epoch

149